<a href="https://colab.research.google.com/github/devdaveddev/DebateBOT/blob/main/DB1Iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import openai
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline, BertForSequenceClassification, BertTokenizer
from textblob import TextBlob
import torch
import random
import numpy as np

# Initialize NLP models
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
summarizer = pipeline("summarization")

# Load BERT model for sentiment analysis
bert_model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def bert_sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = torch.argmax(scores).item()
    return sentiment_score

# OpenAI API Key (Replace with your own)
openai.api_key = "YOUR_OPENAI_API_KEY"

# Function to fetch data from Wikipedia API
def fetch_wikipedia_data(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    response = requests.get(url).json()
    return response.get("extract", "No information found.")

# Function to fetch arguments from Reddit API (requires authentication)
def fetch_reddit_data(topic):
    headers = {"User-Agent": "AI-Debater"}
    url = f"https://www.reddit.com/r/debate/comments.json?q={topic}&sort=top"
    response = requests.get(url, headers=headers).json()
    posts = [post['data']['title'] for post in response['data']['children'][:5]]
    return posts

# Backpropagation-based Weight Adjustment
def update_weights(weights, learning_rate=0.01):
    gradient = np.random.randn(len(weights))  # Simulated gradient update
    new_weights = weights - learning_rate * gradient
    return np.clip(new_weights, 0, 1)  # Ensure weights remain valid

# Function to generate debate arguments using OpenAI, Wikipedia, and Reddit data
def generate_human_like_arguments(topic, debate_type):
    wikipedia_info = fetch_wikipedia_data(topic)
    reddit_info = fetch_reddit_data(topic)

    # Assigning Weights based on Debate Type
    weights = np.array([
        [0.7, 0.2, 0.1],  # Educated: More Wikipedia, less Reddit, least AI
        [0.0, 0.0, 1.0],  # Radical: Only AI-based extreme arguments
        [0.2, 0.7, 0.1]   # Casual: More Reddit, some Wikipedia, little AI
    ])
    weight_labels = {"educated": 0, "radical": 1, "casual": 2}
    w_wiki, w_reddit, w_ai = weights[weight_labels.get(debate_type, 0)]

    # Update weights dynamically using backpropagation technique
    weights[weight_labels.get(debate_type, 0)] = update_weights(weights[weight_labels.get(debate_type, 0)])

    combined_context = f"Wikipedia Info: {wikipedia_info * w_wiki}\n\nReddit Insights: {reddit_info * w_reddit}"

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You generate arguments by merging factual information from Wikipedia, Reddit, and OpenAI-generated insights, ensuring a human-like debate experience."},
            {"role": "user", "content": f"Debate this topic with strong reasoning: {topic}\n\n{combined_context}"}
        ]
    )
    return response["choices"][0]["message"]["content"]

# Function to analyze sentiment & bias using BERT
def analyze_bias(user_input):
    sentiment_score = bert_sentiment_analysis(user_input)
    if sentiment_score >= 4:
        return "Highly Positive (Potential Bias)"
    elif sentiment_score <= 1:
        return "Highly Negative (Potential Bias)"
    return "Neutral"

# Function to simulate debate
def ai_debate():
    debate_type = input("Choose debate style (educated, radical, casual): ")
    topic = input("Enter a topic to debate: ")

    print("\nFetching arguments...\n")

    openai_arguments = generate_human_like_arguments(topic, debate_type)

    print("AI's Arguments:")
    print(openai_arguments)

    print("\nNow, enter your counter-argument:")
    user_argument = input()

    bias_analysis = analyze_bias(user_argument)
    print("\nBias Analysis:", bias_analysis)

    print("\nAI's Rebuttal:")
    rebuttal = generate_human_like_arguments(f"Counter {user_argument}", debate_type)
    print(rebuttal)

if __name__ == "__main__":
    ai_debate()
